# Exercises for Day 1 of the pyMOR Online Course 2020

In [ ]:
from pymor.basic import *
import numpy as np

## Exercise 1: Building a 1D model with output

Apart from 2D models, pyMOR's builtin discretization toolkit also supports 1D problems. In this exercise we will solve a 1D problem using pyMOR and attach an output functional to the `Model`.

### Exercise 1 a)

Try to discretize the boundary value problem

$$
\begin{align}
 \left(-d(x,\mu) \cdot u^\prime(x,\mu)\right)^\prime &= f(x) & x &\in (-1, 1) \\
                                u(-1,\mu) &= 0 \\
                                 u(1,\mu) &= 0
\end{align}
$$

where the source term $f(x)$ and the diffusivity $d(x,\mu)$ are given as

$$
d(x,\mu) = 
\begin{cases}
1     & x < 0 \\
0     & x > 0
\end{cases}
\qquad\text{and}\qquad
d(x,\mu) = 
\begin{cases}
1     & x < 0 \\
e^\mu & x > 0.
\end{cases}
$$

Solve the resulting model for a few parameter values and visualize the solution.

**Hints:**
- Create a `StationaryProblem` to feed into `discretize_stationary_cg`.
- Use `LineDomain` to specify a one-dimensional domain.
- Use `ExpressionFunction` to define $f$ and $d$.

In [ ]:
p = StationaryProblem(
    domain=LineDomain([-1,1]),
    diffusion=ExpressionFunction('1 + (x[..., 0] > 0) * (exp(p[0])-1)',
                                 1, parameters={'p': 1}),
    rhs=ExpressionFunction('x[..., 0] < 0', 1),
)

m, _ = discretize_stationary_cg(p, 1/100)

m.visualize((m.solve(-2), m.solve(1)))

### Exercise 1 b)

Can you define `diffusion` by an equivalent `LincombFunction` such that the parameter only appears in the coefficients? Solve the new model again for a few parameters. Notice that matrices are now only assembled once when the `Model` is created.

**Hints:**
- Use an `ExpressionParameterFunctional` as one of the coefficients.
   

In [ ]:
p = StationaryProblem(
    domain=LineDomain([-1,1]),
    diffusion=LincombFunction(
        [ExpressionFunction('x[..., 0] < 0', 1),
         ExpressionFunction('x[..., 0] > 0', 1)],
        [1,
         ExpressionParameterFunctional('exp(p[0])', {'p': 1})
        ]
    ),
    rhs=ExpressionFunction('x[..., 0] < 0', 1),
)

m, _ = discretize_stationary_cg(p, 1/100)

m.visualize((m.solve(-2), m.solve(1)))

### Exercise 1 c)

In addition, define a linear output functional $\ell$ for the `StationaryProblem` given by the average value of the solution over the domain, i.e.

$$
\ell(u) := \frac{1}{2} \int_{-1}^1 u(x) \,\mathrm{dx}.
$$

Create an output vs. parameter plot for the model for $\mu$ ranging from $-3$ to $3$.

**Hints:**
- Output functionals are specified using the `outputs` parameter of `StationaryProblem.__init__`. 
  The parameter takes a list of 2-tuples of the form `(type, defining_function)` where `type` can either be
  `'l2'` or `'l2_boundary'` depending on whether the functional is given by an $L^2$-inner product type integral
  over the domain or the domain boundary. In our case, we have to specify `'l2'` as type, and the constant
  function with value $1/2$ as `defining_function`.
  
- To compute the output for given parameter values `mu`, use `Model.output`, which returns a `VectorArray` with
  the output quantities. Since we want to access the actual values, we need to call its 'to_numpy()' method
  which, in this case, will return a $1 \times 1$ NumPy array with the desired value.
  To obtain the solution along with the corresponding output, you can use `Model.solve` with `return_output=True`.
  
- Use the `plot` function from `matplotlib.pyplot` to create the plot.

In [ ]:
p = StationaryProblem(
    domain=LineDomain([-1,1]),
    diffusion=LincombFunction(
        [ExpressionFunction('x[..., 0] < 0', 1),
         ExpressionFunction('x[..., 0] > 0', 1)],
        [1,
         ExpressionParameterFunctional('exp(p[0])', {'p': 1})
        ]
    ),
    rhs=ExpressionFunction('x[..., 0] < 0', 1),
    outputs=[('l2', ConstantFunction(0.5, 1))]
)

m, _ = discretize_stationary_cg(p, 1/100)

mus = np.linspace(-3, 3, 20)
outputs = [m.output(mu).to_numpy()[0,0] for mu in mus]

import matplotlib.pyplot as plt
plt.plot(mus, outputs)

## Exercise 2: Solving a time-dependent advection-diffusion equation

So far we have only considered pure diffusion equations. In this exercise will add an advection term and also add time to the equation.

### Exercise 2 a)

Discretize and solve the following boundary value problem for different values of $\mu$:

$$
\begin{align}
- \Delta u(x,y,\mu) + \mu \cdot \nabla \cdot \left(\begin{bmatrix}-y \\ x\end{bmatrix} \cdot u(x,y,\mu)\right) &= f(x,y) & (x,y) &\in \Omega := (-1, 1) \times (-1, 1) \\
u(x,y,\mu) &= 0 & (x,y) &\in \partial\Omega.
\end{align}
$$

The source term $f(x,y)$ is given as

$$
f(x,y) =
\begin{cases}
1 & (x-0.5)^2 + y^2 < 0.01 \\
0 & \text{otherwise}.
\end{cases}
$$

**Hints:**
- Use the `advection` parameter to specify the flux field $[-y, x]^T$.
- Use an `ExpressionFunction` with `shape_range=2` to define the flux field.
- In the expression, you can use all methods of the numpy module in the form `'np.XYZ'`.
- You can use `np.concatenate` to build the vector.
- For parameter separation you can define a `LincombFunction` with a single function and a single coefficient. You can also simply multiply the function with the parameter functional.

In [ ]:
p = StationaryProblem(
    domain=RectDomain([[-1,-1],[1,1]]),
    diffusion=ConstantFunction(1, 2),
    advection=
        ProjectionParameterFunctional('p', 1) *
        ExpressionFunction('np.concatenate([-x[...,1:2], x[...,0:1]], axis=-1)', 2, 2),
    rhs=ExpressionFunction('((x[..., 0] - 0.5)**2 + x[..., 1]**2) < 0.01', 2)
)

m, _ = discretize_stationary_cg(p, 1/100)

m.visualize((m.solve(10), m.solve(100)), separate_colorbars=True)

### Exercise 2 b)

Now solve a non-stationary version of this problem:

$$
\begin{align}
\partial_t u(x,y,t,\mu) - \Delta u(x,y,t,\mu) + \mu \cdot \nabla \cdot \left(\begin{bmatrix}-y \\ x\end{bmatrix} \cdot u(x,y,t,\mu)\right) &= f(x,y) & (x,y) &\in \Omega := (-1, 1) \times (-1, 1),\ t \in (0, 0.1) \\
u(x,y,t,\mu) &= 0 & (x,y) &\in \partial\Omega,\ t \in (0,0.1) \\
u(x,y,0,\mu) &= 0 & (x,y) &\in \Omega. \\
\end{align}
$$

**Hints:**
- Create an `InstationaryProblem` by passing it your `StationaryProblem`, a `ConstantFunction` defining the initial condition and the final simulation time.
- Build the `Model` using `discretize_instationary_cg`. Pass the number of (implicit Euler) time steps using the `nt` parameter.
- `solve` will return a `VectorArray` of all solution time-steps. `visualize` will render the array as a time series.

In [ ]:
p = InstationaryProblem(p, ConstantFunction(0, 2), 0.1)
m, _ = discretize_instationary_cg(p, diameter=1/100, nt=10)
m.visualize(m.solve(100))

## Exercise 3: Unstructured meshes and Robin boundary conditions

So far, we have only defined our problems on rectangular domains for which we could use structured meshes. pyMOR's discretization toolkit also supports unstructured triangle meshes created with Gmsh. These can be directly read using `pymor.discretizers.builtin.grids.gmsh.load_gmsh`. In this exercise, we will use pyMOR `domaindescriptions` that are transformed into a Gmsh geometry definition for meshing.

### Exercise 3 a)

Solve the Poisson equation
$$
\begin{align}
- \Delta u(x) &= f(x) & x &\in \Omega\\
         u(x) &= 0    & x &\in \partial\Omega
\end{align}
$$

where the domain $\Omega$ is the circular sector defined by

$$
\Omega := \left\{
\begin{bmatrix}r\cdot\cos(\phi) \\ r\cdot\sin(\phi) \end{bmatrix} \ \middle|\ 0\leq r < 1,\ 0 \leq \phi < 1.9\cdot\pi
\right\}.
$$

**Hints:**
- Use `CircularSectorDomain` to define $\Omega$.

In [ ]:
p = StationaryProblem(
    domain=CircularSectorDomain(1.9*np.pi, 1),
    diffusion=ConstantFunction(1, 2),
    rhs=ConstantFunction(1,2)
)

m, _ = discretize_stationary_cg(p, 1/10)
m.visualize(m.solve())

### Exercise 3 b)

Solve

$$
\begin{align}
       - \Delta u(x) &= f(x) & x &\in \Omega\\
-\nabla u(x) \cdot n &= -1   & x &\in \partial\Omega \cap \mathbb{R} \times \{0\}\\
                u(x) &= 0    & x &\in \partial\Omega \setminus \mathbb{R} \times \{0\}
\end{align}
$$

on the domain $\Omega$ given by the following heat-sink geometry:

![heatsink.png](heatsink.png)

**Hints:**
- Use `PolygonalDomain` to define $\Omega$.

In [ ]:
def heat_sink_domain(bottom, fins):
    fin_length = 10
    fin_height = 0.1
    fin_space = 0.1
    fin_count  = 10
    base_width = 4
    base_height = 0.2

    points = [[base_width/2, 0],
              [base_width/2, base_height]]

    def add_fin(right):
        lp = points[-1]
        sign = 1 if right else -1
        new_points = [[lp[0],                   lp[1] + sign*0.5*fin_space],
                      [lp[0] + sign*fin_length, lp[1] + sign*0.5*fin_space],
                      [lp[0] + sign*fin_length, lp[1] + sign*(0.5*fin_space + fin_height)],
                      [lp[0],                   lp[1] + sign*(0.5*fin_space + fin_height)],
                      [lp[0],                   lp[1] + sign*(fin_height + fin_space)]]
        points.extend(new_points)

    for _ in range(fin_count):
        add_fin(True)

    points.append([points[-1][0] - base_width, points[-1][1]])

    for _ in range(fin_count):
        add_fin(False)

    points.append([-base_width/2, 0])

    domain = PolygonalDomain(points, {fins: list(range(len(points))),
                                      bottom: [len(points)]})
    return domain

p = StationaryProblem(
    domain=heat_sink_domain('neumann', 'dirichlet'),
    diffusion=ConstantFunction(1, 2),
    neumann_data=ConstantFunction(-1, 2),
)
m, _ = discretize_stationary_cg(p, diameter=1/100)
m.visualize(m.solve())

### Exercise 3 c)

Let's solve a physically somewhat more realistic model by imposing Robin boundary conditions on the fins, i.e., solve

$$
\begin{align}
       - d \cdot \Delta u(x)  &= f(x)                 & x &\in \Omega\\
- d \cdot \nabla u(x) \cdot n &= -80                  & x &\in \partial\Omega \cap \mathbb{R} \times \{0\}\\
- d \cdot \nabla u(x) \cdot n &= 1\cdot(u(x) - 24)    & x &\in \partial\Omega \setminus \mathbb{R} \times \{0\}
\end{align}
$$

with $d = 10^3$ for the same heat-sink domain $\Omega$ as before.

**Hints:**
- Pass the (constant) Robin data functions $1$ and $24$ as a tuple to `StationaryProblem.__init__` via the `robin_data` parameter.

In [ ]:
p = StationaryProblem(
    domain=heat_sink_domain('neumann', 'robin'),
    diffusion=ConstantFunction(1000, 2),
    neumann_data=ConstantFunction(-80, 2),
    robin_data=(ConstantFunction(1, 2), ConstantFunction(24, 2))
)
m, _ = discretize_stationary_cg(p, diameter=1/100)
m.visualize(m.solve()) 